In [67]:
# https://www.nih.gov/news-events/news-releases/nih-clinical-center-provides-one-largest-publicly-available-chest-x-ray-datasets-scientific-community

import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import os
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms as tf
from os import walk
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import image
from torchvision import transforms
# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
train_df = pd.read_csv('train_df2.csv')
test_df = pd.read_csv('test_df2.csv')

In [3]:
train_df.sample(10)

,Finding Label,Image Index,h,w,x,y
28192,1,00008788_000.png,2048.0,2500.0,0.168,0.168
19891,5,00006273_001.png,2048.0,2500.0,0.171,0.171
64862,1,00020170_004.png,2544.0,3056.0,0.139,0.139
48542,13,00014971_006.png,2048.0,2500.0,0.168,0.168
85752,5,00030047_000.png,3056.0,2544.0,0.139,0.139
51076,1,00015806_006.png,2048.0,2500.0,0.168,0.168
79146,1,00026521_000.png,3056.0,2544.0,0.139,0.139
70437,8,00022156_003.png,2544.0,3056.0,0.139,0.139
43103,8,00013287_000.png,2048.0,2500.0,0.168,0.168
22652,6,00007065_002.png,2048.0,2500.0,0.168,0.168


In [4]:
train_df['Finding Label'].value_counts()

1     50500
4      8758
8      8280
5      5361
6      2429
3      2198
12     2056
0      1515
9      1241
13     1122
7       969
11      902
10      868
14      235
2        90
Name: Finding Label, dtype: int64

In [5]:
# (train_df['Image Index'] == '00030181_001.png').value_counts()
# 00028247_001.png

In [6]:
import os
import platform
my_path = "../Datasets/Lungs_Dataset/Xray" if platform.system() == 'Windows' else "datasets/data/images"
filename_list = []
for root, dirs, files in os.walk(my_path, topdown=True):
    for name in files:
        filename_list.append(name)
#         with Image.open(os.path.join("datasets/data/images", name)) as f:
#             print(len(f.size))

In [7]:
train_df['Image Index'].isin(filename_list).value_counts()

True     82995
False     3529
Name: Image Index, dtype: int64

In [8]:
train_df = train_df[train_df['Image Index'].isin(filename_list)]

In [9]:
train_df['Image Index'].isin(filename_list).value_counts()

True    82995
Name: Image Index, dtype: int64

In [10]:
train_df['Image Index'].isna().value_counts()

False    82995
Name: Image Index, dtype: int64

In [11]:
test_df = test_df[test_df['Image Index'].isin(filename_list)]

In [12]:
test_df.sample(10)

,Finding Label,Image Index,h,w,x,y
14810,5,00017606_010.png,2048.0,2500.0,0.168,0.168
7132,8,00010936_012.png,2048.0,2500.0,0.168,0.168
6155,1,00009892_031.png,2500.0,2048.0,0.168,0.168
15990,13,00018867_015.png,2544.0,3056.0,0.139,0.139
15408,1,00018253_030.png,2048.0,2500.0,0.168,0.168
18370,13,00021420_015.png,2048.0,2500.0,0.168,0.168
3791,1,00006271_007.png,2048.0,2500.0,0.171,0.171
2564,1,00004342_031.png,2368.0,3040.0,0.139,0.139
616,6,00000643_008.png,2991.0,2766.0,0.143,0.143
1618,4,00002316_005.png,2657.0,2918.0,0.143,0.143


In [13]:
class RescaleImage(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
            
        The samples coming into this class will have its images reduced assuming
        the input is a h, w, c numpy array
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        h, w = image.size

        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = image.resize((new_h, new_w))

        return {'image': img, 'label': label}

In [14]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        image = np.array(image)
        if len(image.shape) > 2:
            image = image[:,:,0]

        # numpy image: H x W x C
        # torch image: C X H X W
        return {'image': torch.FloatTensor(image).unsqueeze(0),
#         return {'image': torch.from_numpy(image),                
                'label': torch.from_numpy(np.array(label))}

In [15]:
#  print([Image.open('../Datasets/Lungs_Dataset/Xray/'+train_df['Image Index'][each_import]).size for each_import in range(100)])

In [66]:
#Note this will return an Image object, of h and w, and its corresponding label
class CovidLungsDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, dataframe, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.dataframe['Image Index'][idx])
        my_image = Image.open(img_name)        
        if len(my_image.size) > 2:
            assert len(my_image.size) > 2
        row = self.dataframe.iloc[idx]
        label = row['Finding Label']
        sample = {'image': my_image, 'label': label}
        sample.to(device)
        if self.transform:
            sample = self.transform(sample)

        return sample

In [17]:
my_train_set = CovidLungsDataset(train_df, my_path, transform=transforms.Compose([
        RescaleImage(200),
        ToTensor()
]))

In [18]:
'''The original data coming out of the Dataset is a dictionary, 
the image is an Image object with the corresponding label.
The size of all of these images are 1024 by 1024.
The RescaleImage will convert the Image object to an Image object with 200x200 in size, 
leaving the label alone.
The ToTensor will convert the Image object with 200x200 to tensors of 1x200x200'''
from random import randint
a_sample = my_train_set.__getitem__(randint(0,256))
my_image = a_sample['image']
my_label = a_sample['label']
#imshow will work if it's h,w,c or h,w
#torch is c,h,w
#line below can be used if you don't use ToTensor
# plt.imshow(my_image)
type(my_image), my_image.shape, my_label

(torch.Tensor, torch.Size([1, 200, 200]), tensor(11))

In [19]:
# plt.imshow(my_image.squeeze(0));

In [29]:
import platform
batch_loader_params = {
    "batch_size": 5,
    "shuffle": True,
    "num_workers": 0 if platform.system() == 'Windows' else 2
}
dataloader = DataLoader(my_train_set, **batch_loader_params)

In [21]:
# iter(dataloader).next()
# for i, each in enumerate(dataloader):
#     print(each['image'].shape, each['label'].shape)

In [22]:
# batch_samples = iter(dataloader)
# samples = batch_samples.next()
# datset_batch = torchvision.utils.make_grid(samples['image'])

In [23]:
# plt.figure(figsize=(20,10))
# for index, each in enumerate(datset_batch):
#     plt.imshow(each.squeeze(0))

In [24]:
def spatial_size(input_size: int, kernel_size: int, stride: int = 1, padding: int = 0):
    # https://cs231n.github.io/convolutional-networks/
    spatial_size = (input_size - kernel_size + 2 * padding)/stride + 1
    assert spatial_size % 1 == 0
    assert spatial_size > 0
    return int(spatial_size)

In [48]:
print(spatial_size(200, 51))

150


In [49]:
print(spatial_size(75, 50))

26


In [51]:
print(spatial_size(13, 8))

6


In [65]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 200, 51, 1)
        self.conv2 = nn.Conv2d(200, 75, 50, 1)        
        self.conv3 = nn.Conv2d(75, 13, 8, 1)  
        self.fc1 = nn.Linear(3*3*13, 32)
        self.fc2 = nn.Linear(32, 15)

    def forward(self, x):
        x = F.relu(self.conv1(x)) # torch.Size([5, 1, 200, 200]) ==> torch.Size([5, 200, 150, 150])
        x = F.max_pool2d(x, 2, 2) # torch.Size([5, 200, 150, 150]) ==> torch.Size([5, 200, 75, 75])
        x = F.relu(self.conv2(x)) # torch.Size([5, 200, 75, 75]) ==> torch.Size([5, 75, 26, 26])
        x = F.max_pool2d(x, 2, 2) # torch.Size([5, 75, 26, 26]) ==> torch.Size([5, 75, 13, 13])
        x = F.relu(self.conv3(x)) # torch.Size([5, 75, 13, 13]) ==> torch.Size([5, 13, 6, 6])
        x = F.max_pool2d(x, 2, 2) # torch.Size([5, 13, 6, 6]) ==> torch.Size([5, 13, 3, 3])
        x = x.view(-1, 3*3*13)  # torch.Size([5, 13, 3, 3]) ==> torch.Size([5*32, 3*3*13])
        x = F.relu(self.fc1(x))   # torch.Size([5*32, 3*3*13]) ==> torch.Size([5, 32])
        x = self.fc2(x)           # torch.Size([5, 32]) ==> torch.Size([5, 15])
        # # There's no activation at the final layer because of the criterion of CEL
#         return x
        return torch.log_softmax(x, dim=-1)


net = Net()
net.to(device)
print(net)

Net(
  (conv1): Conv2d(1, 200, kernel_size=(51, 51), stride=(1, 1))
  (conv2): Conv2d(200, 75, kernel_size=(50, 50), stride=(1, 1))
  (conv3): Conv2d(75, 13, kernel_size=(8, 8), stride=(1, 1))
  (fc1): Linear(in_features=117, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=15, bias=True)
)


In [62]:
# criterion = nn.CrossEntropyLoss()
criterion = nn.NLLLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [63]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data['image'], data['label']
        
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(data['image'])
#         print(outputs.shape, data['label'].shape)
        loss = criterion(outputs, data['label'])
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        print('[%d, %5d] loss: %.5f' %
              (epoch + 1, i + 1, running_loss / (epoch*i+1)))

[1,     1] loss: 4.06250
[1,     2] loss: 4470536.56250
[1,     3] loss: 5154667.75000
[1,     4] loss: 5197298.67578
[1,     5] loss: 5197300.80633
[1,     6] loss: 5470878.18133
[1,     7] loss: 5470947.38091
[1,     8] loss: 5621659.80278
[1,     9] loss: 5621662.39446
[1,    10] loss: 5621665.01035
[1,    11] loss: 5621667.57748
[1,    12] loss: 5621670.08818
[1,    13] loss: 5621672.61248
[1,    14] loss: 5621675.11744
[1,    15] loss: 5621677.58928
[1,    16] loss: 5621680.10285
[1,    17] loss: 5621682.60778
[1,    18] loss: 5621684.99020
[1,    19] loss: 5621687.33970
[1,    20] loss: 5621689.68018
[1,    21] loss: 5621692.07051
[1,    22] loss: 5621694.49949
[1,    23] loss: 5621696.90794
[1,    24] loss: 5621699.05175
[1,    25] loss: 5621701.30669
[1,    26] loss: 5621703.66442
[1,    27] loss: 5621706.02275
[1,    28] loss: 5621708.15130
[1,    29] loss: 5621710.34438
[1,    30] loss: 5621712.87823
[1,    31] loss: 5621714.91864
[1,    32] loss: 5621717.36402
[1,    33] los

[1,   266] loss: 5622097.21058
[1,   267] loss: 5622099.32780
[1,   268] loss: 5622100.91128
[1,   269] loss: 5622102.60901
[1,   270] loss: 5622104.37409
[1,   271] loss: 5622106.09053
[1,   272] loss: 5622108.62552
[1,   273] loss: 5622109.22879
[1,   274] loss: 5622111.21660
[1,   275] loss: 5622113.20691
[1,   276] loss: 5622113.81174
[1,   277] loss: 5622115.08026
[1,   278] loss: 5622116.76002
[1,   279] loss: 5622117.90542
[1,   280] loss: 5622118.82009
[1,   281] loss: 5622120.35297
[1,   282] loss: 5622121.59039
[1,   283] loss: 5622123.40580
[1,   284] loss: 5622125.88912
[1,   285] loss: 5622126.44443
[1,   286] loss: 5622127.33756
[1,   287] loss: 5622128.75253
[1,   288] loss: 5622131.22701
[1,   289] loss: 5622132.95340
[1,   290] loss: 5622134.18624
[1,   291] loss: 5622135.05531
[1,   292] loss: 5622137.36475
[1,   293] loss: 5622138.22543
[1,   294] loss: 5622139.09918
[1,   295] loss: 5622141.78978
[1,   296] loss: 5622143.02544
[1,   297] loss: 5622146.47027
[1,   29

Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/senhmo/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/Users/senhmo/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/senhmo/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/senhmo/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/Users/senhmo/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/senhmo/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/senhmo/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeErro

KeyboardInterrupt: 